In [ ]:
import random
import tkinter as tk
from tkinter import messagebox
from PIL import ImageTk, Image
import speech_recognition as sr
import cv2
import mediapipe as mp

# Create the main window
root = tk.Tk()
background_label = tk.Label(root)
background_label.grid(row=0, column=0, columnspan=2, rowspan=16)
root.title("Relief Operation Game")

# Game variables
resources = 100  # Initial resources
people = 50  # Initial number of survivors
day = 1  # Initial day of the relief operation
rescue_teams = 3  # Initial number of rescue teams
supplies = 50  # Initial amount of supplies
injured_people = 0  # Initial number of injured people
morale = 100  # Initial morale level
damage = 0  # Initial damage level
weather = 'Sunny'  # Initial weather conditions
game_duration = 14  # Total duration of the game in days
days_to_rescue = 7  # Initial number of days remaining to rescue survivors

# Function to update game state
def update_game_state():
    global resources, people, injured_people, morale, damage, weather, day,  days_to_rescue
    resources -= 10  # Resources consumed for daily operations
    people -= random.randint(5, 10)  # People lost due to the tsunami
    injured_people += random.randint(0, 5)  # Randomly generate number of injured people
    morale -= 1  # Morale decreases each day
    damage += random.randint(1, 5)  # Randomly increase damage level
    days_to_rescue -= 1  # Reduce days remaining to rescue survivors
    # Update weather conditions
    weather = random.choice(['Sunny', 'Cloudy', 'Rainy', 'Stormy'])
    update_gui()

# Function to print game state
# Function to print game state and update GUI
def update_gui():
    weather_icons = {
        'Sunny': 'Sunny ☀️',
        'Cloudy': 'Cloudy 🌥️',
        'Rainy': 'Rainy 🌧️',
        'Stormy': 'Stormy ⛈️'
    }
    # Clear previous game state
    for widget in root.grid_slaves():
        widget.grid_forget()

    # Update game state in GUI
    tk.Label(root, text="Day: " + str(day)).grid(row=0, column=0, sticky="w")
    tk.Label(root, text="Resources: " + str(resources)).grid(row=1, column=0, sticky="w")
    tk.Label(root, text="People: " + str(people)).grid(row=2, column=0, sticky="w")
    tk.Label(root, text="Rescue Teams: " + str(rescue_teams)).grid(row=3, column=0, sticky="w")
    tk.Label(root, text="Supplies: " + str(supplies)).grid(row=4, column=0, sticky="w")
    tk.Label(root, text="Injured People: " + str(injured_people)).grid(row=5, column=0, sticky="w")
    tk.Label(root, text="Morale: " + str(morale)).grid(row=6, column=0, sticky="w")
    tk.Label(root, text="Damage: " + str(damage)).grid(row=7, column=0, sticky="w")
    tk.Label(root, text="Weather: " + weather_icons[weather]).grid(row=8, column=0, sticky="w")
    tk.Label(root, text="Days remaining to rescue survivors: " + str(days_to_rescue)).grid(row=9, column=0, sticky="w")
    

    # Change background image based on weather condition
    if weather == 'Sunny':
        background_image = tk.PhotoImage(file="C:/Users/harih/Documents/College-AIE/Code at Night/sunny.png")
    elif weather == 'Cloudy':
        background_image = tk.PhotoImage(file="C:/Users/harih/Documents/College-AIE/Code at Night/cloudy.png")
    elif weather == 'Rainy':
        background_image = tk.PhotoImage(file="C:/Users/harih/Documents/College-AIE/Code at Night/rainy.png")
    elif weather == 'Stormy':
        background_image = tk.PhotoImage(file="C:/Users/harih/Documents/College-AIE/Code at Night/stormy.png")
    else:
        background_image = tk.PhotoImage(file="C:/Users/harih/Documents/College-AIE/Code at Night/default.png")

    # Update background image in GUI
    background_label.config(image=background_image)
    background_label.image = background_image
    background_label.grid(row=0, column=0, columnspan=2, rowspan=16)
    if resources <= 0 or people <= 0 or morale <= 0 or days_to_rescue <= 0:
        game_over()
    # Add buttons for game actions with evenly spaced columnspan
    tk.Button(root, text="Send Rescue Team", command=send_rescue_team).grid(row=11, column=0, columnspan=2, sticky="we")
    tk.Button(root, text="Distribute Supplies", command=distribute_supplies).grid(row=12, column=0, columnspan=2, sticky="we")
    tk.Button(root, text="Treat Injured", command=treat_injured).grid(row=13, column=0, columnspan=2, sticky="we")
    tk.Button(root, text="Repair Damage", command=repair_damage).grid(row=14, column=0, columnspan=2, sticky="we")
    tk.Button(root, text="Next Day", command=next_day).grid(row=16, column=0, columnspan=2, sticky="we")
    tk.Button(root, text="Hint", command=provide_hint).grid(row=15, column=0, columnspan=2, sticky="we")
    tk.Button(root, text="Voice Command", command=handle_voice_command).grid(row=17, column=0, columnspan=2, sticky="we")
    tk.Button(root, text="Image Recognition", command=Couning_fingers).grid(row=18, column=0, columnspan=2, sticky="we")
    tk.Button(root, text="Quit", command=quit_game).grid(row=19, column=0, columnspan=2, sticky="we")
    # Set column weight to make buttons evenly spaced out
    root.grid_columnconfigure(0, weight=1)

    # Update the GUI
    root.update()
    

# Function to handle voice commands
def handle_voice_command():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for voice command...")
        audio = r.listen(source)
    try:
        # Use Google Web Speech API to recognize voice input
        command = r.recognize_google(audio)
        print("Voice command: ", command)
        # Map voice input to corresponding GUI option
        if "send rescue team" in command.lower():
            send_rescue_team()
        elif "distribute supplies" in command.lower():
            distribute_supplies()
        elif "treat injured" in command.lower():
            treat_injured()
        elif "repair damage" in command.lower():
            repair_damage()
        elif "hint" in command.lower():
            provide_hint()
        elif "next day" in command.lower():
            next_day()
        else:
            print("Voice command not recognized.")
    except sr.UnknownValueError:
        print("Could not understand voice input.")
    except sr.RequestError as e:
        print("Error retrieving voice input: ", e)

    
def game_over():
    messagebox.showinfo("Game Over", "The relief operation has failed. You were unable to rescue all the survivors in time.")
    root.destroy()

def send_rescue_team():
    global resources, rescue_teams, days_to_rescue
    if rescue_teams > 0:
        
        rescued = random.randint(1, 10)
        resources -= rescued * 5 # Resources consumed for sending rescue team
        people += rescued
        days_to_rescue += 1
        rescue_teams -= 1
        update_gui()
    else:
        messagebox.showinfo("Error", "No rescue teams available!") 

def distribute_supplies():
    global resources, supplies
    if supplies > 0:
        distributed = random.randint(1, 10)
        resources -= distributed * 2 # Resources consumed for distributing supplies
        morale += distributed
        supplies -= distributed
        update_gui()
    else:
        messagebox.showinfo("Error", "No supplies available!")

def treat_injured():
    global resources, injured_people
    if injured_people > 0:
        treated = random.randint(1, 5)
        resources -= treated * 3 # Resources consumed for treating injured
        injured_people -= treated
        update_gui()
    else:
        messagebox.showinfo("Error", "No injured people to treat!")
        
def repair_damage():
    global resources, damage
    if damage > 0:
        repaired = random.randint(1, 5)
        resources -= repaired * 4 # Resources consumed for repairing damage
        damage -= repaired
        update_gui()
    else:
        messagebox.showinfo("Error", "No damage to repair!")


def next_day():
    global day
    update_game_state()
    day += 1
    if day >= game_duration:
        game_over()

# Function to provide hint to the player
def provide_hint():
    global resources, people, injured_people, morale, damage, weather, day, days_to_rescue
    hint = "Hints:\n\n"
    if resources <= 0:
        hint += "Low resources! Try sending rescue teams to gather more supplies.\n"
    if people <= 0:
        hint += "Low number of survivors! Try rescuing more survivors or treating injured people.\n"
    if morale <= 0:
        hint += "Low morale! Try distributing supplies or treating injured people to boost morale.\n"
    if days_to_rescue <= 0:
        hint += "Time's running out! Rescue more survivors as quickly as possible.\n"
    if damage > 0:
        hint += "High damage! Try repairing damage to prevent further losses.\n"
    if injured_people > 0:
        hint += "Injured people! Try treating injured people to prevent further casualties.\n"
    messagebox.showinfo("Hint", hint)

import cv2
import mediapipe as mp

def Couning_fingers():
    cap = cv2.VideoCapture(0)
    mp_Hands = mp.solutions.hands
    hands = mp_Hands.Hands()
    mpDraw = mp.solutions.drawing_utils
    finger_Coord = [(8, 6), (12, 10), (16, 14), (20, 18)]
    thumb_Coord = (4,2)
    while True:
        success, image = cap.read()
        RGB_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(RGB_image)
        multiLandMarks = results.multi_hand_landmarks
        if multiLandMarks:
            handList = []
            for handLms in multiLandMarks:
                mpDraw.draw_landmarks(image, handLms, mp_Hands.HAND_CONNECTIONS)
                for idx, lm in enumerate(handLms.landmark):
                    h, w, c = image.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    handList.append((cx, cy))
                for point in handList:
                    cv2.circle(image, point, 10, (255, 255, 0), cv2.FILLED)
                upCount = 0
                for coordinate in finger_Coord:
                    if handList[coordinate[0]][1] < handList[coordinate[1]][1]:
                        upCount += 1
                if handList[thumb_Coord[0]][0] > handList[thumb_Coord[1]][0]:
                    upCount += 1
                cv2.putText(image, str(upCount), (150,150), cv2.FONT_HERSHEY_PLAIN, 12, (0,255,0), 12)
            cv2.imshow("Counting number of fingers", image)
            cv2.waitKey(1)
            if (upcount == 1) in command.lower():
                send_rescue_team()
            elif (upcount == 2) in command.lower():
                distribute_supplies()
            elif (upcount == 3) in command.lower():
                treat_injured()
            elif (upcount == 4) in command.lower():
                repair_damage()
            elif (upcount == 5) in command.lower():
                next_day()
            else: 
                provide_hint()
                cap.release()
                cv2.destroyAllWindows()


def quit_game():
    if messagebox.askyesno("Quit", "Are you sure you want to quit the game?"):
        root.destroy()


update_gui()
root.mainloop()